# GENERALS

In [ ]:
%pip install -qU langchain langchain_community langchain_core

In [264]:
# GENERAL IMPORTS
import os
import re
from langchain.schema import Document
from rich import print
from tqdm import tqdm
from typing import List

In [242]:
# RICH'S PRINT COLORS
YELLOW = "#fde047"
ORANGE = "#f97316"
RED = "#ef4444"
BLUE = "#3b82f6"
CYAN = "#06b6d4"
EMERALD = "#34d399"
VIOLET = "#a855f7"
PINK = "#ec4899"
GRAY = "#64748b"
WHITE = "#cccccc"
GREEN = "#3fb618"

In [ ]:
# GENERAL VARIABLES
ROOT_DIR = "../../../COLEGA DATA"
PDF_DIR = os.path.join(ROOT_DIR, "notificaciones")
# Ruta al archivo PDF
PDF_FILE_1 = os.path.join(PDF_DIR, "RES 04-04-2024 - DILIGENCIA PRELIMINAR.pdf")
PDF_FILE_2 = os.path.join(
    ROOT_DIR, "MÉTODO DE LA DEMANDA Y SU CONTESTACIÓN/1_EL_CASO_Y_SU_SOLUCIÓN.pdf"
)

print(f"[{WHITE}]{PDF_DIR}\n\n{PDF_FILE_1}\n\n{PDF_FILE_2}[/]")

In [192]:
def text_cleaner(text: str) -> str:
    """
    Cleans text by replacing non-breaking spaces, normalizing spaces and newlines,
    and removing hash symbols.
    """
    try:
        # Replace non-breaking spaces with regular spaces
        text = text.replace("\xa0", " ")
        # Normalize spaces
        text = re.sub(r"\s+", " ", text)
        # Normalize newlines if specified
        text = re.sub(r"\n{3,}", "\n\n", text)
        # Remove hash symbols if specified
        text = re.sub(r"#", "", text)
        # Trim leading and trailing whitespace
        text = text.strip()

        return text
    except Exception as e:
        print(f"An error occurred while cleaning the text: {e}")
        return text

---
---
---

# PYMUPDF4LLM

In [ ]:
%pip install -qU pymupdf4llm

In [198]:
from pymupdf4llm import to_markdown

In [204]:
def pymupdf4llm_directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"pymupdf4llm_directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append({"file_name": filename, "file_path": file_path})

    # LOADS EACH PDF FILE: FILE --> LIST[DOCUMENT]
    for file_info in tqdm(
        pdf_files_info,
        desc="LOADING PDF FILES",
        total=len(pdf_files_info),
        colour=EMERALD,
    ):
        md_text = to_markdown(file_info["file_path"], show_progress=False)
        loaded_file = Document(metadata=file_info, page_content=md_text)

        loaded_docs.append(loaded_file)

    return loaded_docs

In [ ]:
pymupdf4llm_docs = pymupdf4llm_directory_loader(PDF_DIR)

In [ ]:
len(pymupdf4llm_docs)

In [ ]:
for index, doc in enumerate(pymupdf4llm_docs):
    print(
        f"[bold {BLUE}]> DOC N°:[/] [bold {WHITE}]{index}[/]\n",
        f"[bold {EMERALD}]> FILENAME:[/] [bold {WHITE}]{doc.metadata["file_name"]}[/]\n\n",
        f"[bold {YELLOW}]> CONTENT:[/]\n[{WHITE}]{doc.page_content}[/]",
    )

---
---
---

# PYTESSARACT / PDF2IMAGE

In [ ]:
%pip install -qU pytesseract pdf2image poppler-utils

In [ ]:
import pytesseract
from pdf2image import convert_from_path
from tqdm import tqdm

In [228]:
def pytesseract_directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"pymupdf4llm_directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append({"file_name": filename, "file_path": file_path})

    # CONVERTS EACH PDF FILE INTO A LIST[PNG]
    for file_info in tqdm(
        pdf_files_info,
        desc="LOADING PDF FILES",
        total=len(pdf_files_info),
        colour=EMERALD,
    ):
        pages_imgs = convert_from_path(file_info["file_path"])
        pages = []
        for page in pages_imgs:
            page_extracted_text = pytesseract.image_to_string(page, lang="spa")
            pages.append(page_extracted_text)

        content = "\n".join(page for page in pages)

        loaded_file = Document(metadata=file_info, page_content=content)
        loaded_docs.append(loaded_file)

    return loaded_docs

In [ ]:
pytesseract_docs = pytesseract_directory_loader(PDF_DIR)

In [ ]:
len(pytesseract_docs)

In [ ]:
for index, doc in enumerate(pytesseract_docs):
    print(
        f"[bold {BLUE}]> DOC N°:[/] [bold {WHITE}]{index}[/]\n",
        f"[bold {EMERALD}]> FILENAME:[/] [bold {WHITE}]{doc.metadata["file_name"]}[/]\n\n",
        f"[bold {YELLOW}]> CONTENT:[/]\n[{WHITE}]{doc.page_content}[/]",
    )

---
---
---

# MARKER-PDF

<span style="color: #f97316; font-weight: bold">!TODO: </span>CHECK OUT WHATS GOING ON HERE

<span style="color: #64748b">RuntimeError: MPS backend out of memory (MPS allocated: 9.06 GB, other allocations: 3.52 MB, max allowed: 9.07 GB). Tried to allocate 2.50 KB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).</span>


In [ ]:
%pip install -qU marker-pdf

In [237]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

In [251]:
def marker_directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"marker_directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append({"file_name": filename, "file_path": file_path})

    # LOADS EACH PDF FILE: FILE --> LIST[DOCUMENT]
    for file_info in tqdm(
        pdf_files_info,
        desc="LOADING PDF FILES",
        total=len(pdf_files_info),
        colour=EMERALD,
    ):
        converter = PdfConverter(
            artifact_dict=create_model_dict(),
        )

        rendered = converter(file_info["file_path"])
        text, _, images = text_from_rendered(rendered)
        loaded_file = Document(metadata=file_info, page_content=text)

        loaded_docs.append(loaded_file)

    return loaded_docs

In [ ]:
marker_docs = marker_directory_loader(PDF_DIR)

---
---
---

# SURYA

In [ ]:
%pip install -qU pdf2image surya-ocr

In [261]:
from pdf2image import convert_from_path
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor

In [ ]:
# UNFINISHED
def pytesseract_directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"pymupdf4llm_directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append({"file_name": filename, "file_path": file_path})

    langs = ["es", "en"]

    recognition_predictor = RecognitionPredictor()
    detection_predictor = DetectionPredictor()
    # CONVERTS EACH PDF FILE INTO A LIST[PNG]
    for file_info in tqdm(
        pdf_files_info,
        desc="LOADING PDF FILES",
        total=len(pdf_files_info),
        colour=EMERALD,
    ):
        pages_imgs = convert_from_path(file_info["file_path"])
        pages = []
        for page in pages_imgs:
            page_extracted_text = pytesseract.image_to_string(page, lang="spa")
            pages.append(page_extracted_text)

        content = "\n".join(page for page in pages)

        loaded_file = Document(metadata=file_info, page_content=content)
        loaded_docs.append(loaded_file)

    return loaded_docs

In [ ]:
pages = convert_from_path(PDF_FILE_1)

langs = ["es", "en"]

recognition_predictor = RecognitionPredictor()
detection_predictor = DetectionPredictor()

predictions_per_page = [
    recognition_predictor([page], [langs], detection_predictor) for page in pages
]

In [47]:
# Iterate through each page
txt = ""
for prediction in predictions_per_page:
    for ocr_result in prediction:
        for text_line in ocr_result.text_lines:
            txt += f"\n{text_line.text}"

---
---
---

# TROCOCR
<span style="color: #fde047; font-weight: bold; font-size: x-large">UNTESTED</span>

In [ ]:
%pip install -qU transformers

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

---
---
---

# DOCTR
<span style="color: #fde047; font-weight: bold; font-size: x-large">UNTESTED</span>

In [ ]:
%pip install -qU python-doctr "python-doctr[tf]" "python-doctr[torch]"

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

model = ocr_predictor(
    det_arch="db_resnet50", reco_arch="crnn_vgg16_bn", pretrained=True
)

# read file
img = DocumentFile.from_images(img_path1)

# use pre-trained model
result = model(img)

# export the result as a nested dict
extract_info = result.export()

---
---
---

# PYOCR

In [ ]:
%pip install -qU pyocr

In [96]:
import pyocr
import pyocr.builders

# Ruta al archivo PDF
pdf_path = (
    "../../../COLEGA DATA/notificaciones/RES 04-04-2024 - DILIGENCIA PRELIMINAR.pdf"
)
# pdf_path = "../../../COLEGA DATA/MÉTODO DE LA DEMANDA Y SU CONTESTACIÓN/1_EL_CASO_Y_SU_SOLUCIÓN.pdf"

pages = convert_from_path(pdf_path)

tools = pyocr.get_available_tools()
tool = ValueError("No tools found") if len(tools) == 0 else tools[0]
langs = tool.get_available_languages()
lang = ValueError("'spa' is not available") if "spa" not in langs else "spa"

loaded_pages = []
for page in pages:
    txt: str = tool.image_to_string(
        page, lang="spa", builder=pyocr.builders.TextBuilder()
    )
    loaded_pages.append(txt)

In [ ]:
print("\n\n".join(loaded_pages))